In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
import gensim.models as Fasttext  # For FastText embeddings
from gensim.models.fasttext import load_facebook_model
import numpy as np

In [25]:
data = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")


In [26]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.2)


In [27]:
tokenizer = Tokenizer(num_words=10000)  # Adjust max vocabulary size as needed
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [28]:
max_length = 100  # Adjust maximum post length as needed
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding="post")

In [29]:
import fasttext.util
from urllib.request import urlopen

fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")

In [30]:
vocab_size = 10000  # Adjust to match your vocabulary size
embedding_dim = 300  # Dimensions of the FastText model
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [33]:
#label encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [34]:
# Define the CNN model architecture
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Single output neuron for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the prepared data
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
5800/5802 [============================>.] - ETA: 0s - loss: 0.2516 - accuracy: 0.9005WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001BBB2075CF0> and will run it as-is.
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000001BBB2075CF0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000001BBB2075CF0>. Note that functions defined in certain environments, like the int

In [ ]:
print(type(y_train[0]))

<class 'str'>


In [38]:
# After model training:
model.save('D:\Mini Project\minipj\models\CNN_model1.h5')  # Replace with your desired path and filename
